In [1]:
#load up the basics
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

#Set GPU to use
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#import TF stuff
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Check TensorFlow Version
print('TF version: {}' .format(tf.__version__))

#Check for GPU utilization
if tf.test.gpu_device_name():
    print(' GPU: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU")

TF version: 2.9.1
 GPU: /device:GPU:0


In [2]:
#Load up the TF model

#model path
model_path = './model/FRF_mar15_remap_fullmodel_model'

# load model into tf
model = tf.keras.models.load_model(model_path, compile = True)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 7)  91          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 7)  28         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

 activation_6 (Activation)      (None, 128, 128, 28  0           ['batch_normalization_6[0][0]']  
                                )                                                                 
                                                                                                  
 activation_7 (Activation)      (None, 128, 128, 28  0           ['batch_normalization_7[0][0]']  
                                )                                                                 
                                                                                                  
 conv2d_7 (Conv2D)              (None, 128, 128, 28  63532       ['activation_6[0][0]']           
                                )                                                                 
                                                                                                  
 add_2 (Add)                    (None, 128, 128, 28  0           ['activation_7[0][0]',           
          

                                                                                                  
 activation_15 (Activation)     (None, 32, 32, 112)  0           ['batch_normalization_15[0][0]'] 
                                                                                                  
 conv2d_16 (Conv2D)             (None, 32, 32, 112)  1016176     ['activation_15[0][0]']          
                                                                                                  
 up_sampling2d (UpSampling2D)   (None, 64, 64, 112)  0           ['conv2d_16[0][0]']              
                                                                                                  
 concatenate (Concatenate)      (None, 64, 64, 168)  0           ['up_sampling2d[0][0]',          
                                                                  'add_3[0][0]']                  
                                                                                                  
 batch_nor

                                )                                                                 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 256, 256, 28  1988        ['concatenate_2[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_23 (BatchN  (None, 256, 256, 28  112        ['conv2d_23[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 batch_normalization_24 (BatchN  (None, 256, 256, 28  112        ['conv2d_25[0][0]']              
 ormalization)                  )                                                                 
          

In [3]:
#define the number of classes of the image...

num_classes = 4

#this is a fn to make sure the doodler-derived labels go from 0- (num_classes-1)

def D_Filter(image):
    
    image[image == (2)] = (1) #class 2 -> 1
    image[image == (4)] = (1) #class 4 -> 1
    
    image[image == (3)] = (2) #class 3 -> 2
    image[image == (5)] = (3) #class 3 -> 2
    
    
    image[image>(5)] = (0)
    return image

def standardize(img):
    #standardization using adjusted standard deviation

    N = np.shape(img)[0] * np.shape(img)[1]
    s = np.maximum(np.std(img), 1.0/np.sqrt(N))
    m = np.mean(img)
    img = (img - m) / s
    del m, s, N
    #
    if np.ndim(img)==2:
        img = np.dstack((img,img,img))

    return img


In [4]:
#test the model

#define test directory
test_dir_im = '../data/TVdata/images'
test_dir_lab = '../data/TVdata/labels'

#load images
i_test = []
i_images = []
for img in sorted(os.listdir(test_dir_im)):
    #pull the name of the image
    print(img)
    i_images.append(img)
    #pull the image
    img = os.path.join(test_dir_im, img)
    img = tf.keras.preprocessing.image.load_img(img,target_size = (512,512))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = standardize(img)
    
    img = np.expand_dims(img,axis=0)
#     i_test = img
    
    i_test.append(img)
    
#load labels
l_test = []
l_images = []
for img in sorted(os.listdir(test_dir_lab)):
    #pull the name of the image
    l_images.append(img)
    #pull the image
    img = os.path.join(test_dir_lab, img)
    img = tf.keras.preprocessing.image.load_img(img,target_size = (512,512),color_mode = "grayscale")
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = D_Filter(img)
    img = tf.keras.utils.to_categorical(img, num_classes=4)
    
    img = np.expand_dims(img,axis=0)
#     l_test = img
    
    l_test.append(img)
    
#stack
i_test = np.vstack(i_test)
l_test = np.vstack(l_test)

FRF_c1_snap_20150221160000_EBG.jpg
FRF_c1_snap_20150226160000_EBG.jpg
FRF_c1_snap_20150228160000_EBG.jpg
FRF_c1_snap_20150322160000_EBG.jpg
FRF_c1_snap_20150503160000_EBG.jpg
FRF_c1_snap_20150505160000_EBG.jpg
FRF_c1_snap_20150507160000_EBG.jpg
FRF_c1_snap_20150508160000_EBG.jpg
FRF_c1_snap_20150509160000_EBG.jpg
FRF_c1_snap_20150510160000_EBG.jpg
FRF_c1_snap_20150511160000_EBG.jpg
FRF_c1_snap_20150512160000_EBG.jpg
FRF_c1_snap_20150520160000_EBG.jpg
FRF_c1_snap_20150524160000_EBG.jpg
FRF_c1_snap_20150525160000_EBG.jpg
FRF_c1_snap_20150710160000_EBG.jpg
FRF_c1_snap_20150801_EBG.jpg
FRF_c1_snap_20150901_EBG.jpg
FRF_c1_snap_20151001_EBG.jpg
FRF_c1_snap_20151029160000_EBG.jpg
FRF_c1_snap_20151101_EBG.jpg
FRF_c1_snap_20160101_EBG.jpg
FRF_c1_snap_20160201_EBG.jpg
FRF_c1_snap_20160215160000_EBG.jpg
FRF_c1_snap_20160217160000_EBG.jpg
FRF_c1_snap_20160301_EBG.jpg
FRF_c1_snap_20160306160000_EBG.jpg
FRF_c1_snap_20160401_EBG.jpg
FRF_c1_snap_20160412160000_EBG.jpg
FRF_c1_snap_20160501_EBG.jpg
FRF_

In [5]:
np.shape(i_test)

(107, 512, 512, 3)

In [6]:
nclasses = 4
for index in range(nclasses):
    print(index)

0
1
2
3


In [7]:
#Doodleverse metrics, 

#define new Dice coeff
def E_dice_coef(y_true, y_pred):
    smooth = 10e-6
    y_true_f = tf.reshape(tf.dtypes.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.dtypes.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return dice

#define new multi-dice workflow
def E_dice_multi_metric(nclasses):
    
    def dice_coef(y_true, y_pred):
        dice = 0
        #can't have an argmax in a loss
        #y_pred = tf.one_hot(tf.argmax(y_pred, -1), 4)
        for index in range(nclasses):
            dice += E_dice_coef(y_true[:,:,:,index], y_pred[:,:,:,index])
        return dice/nclasses
    
    return dice_coef

#define the loss, which calls the metrics
def E_dice_coef_loss(nclasses):
    
    def E_dice_MC_coef_loss(y_true, y_pred):
        dice = 0
        #can't have an argmax in a loss
        #y_pred = tf.one_hot(tf.argmax(y_pred, -1), 4)
        for index in range(nclasses):
            dice += E_dice_coef(y_true[:,:,:,index], y_pred[:,:,:,index])
        return 1 - (dice/nclasses)
    
    return E_dice_MC_coef_loss



def new_iou(y_true, y_pred):
    smooth = 10e-6
    y_true_f = tf.reshape(tf.dtypes.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.dtypes.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    union =  tf.reduce_sum(y_true_f + y_pred_f) - intersection
    return (intersection+smooth)/(union+smooth)

def E_iou(nclasses):
    
    def mean_iou(y_true, y_pred):
        iousum = 0
        y_pred = tf.one_hot(tf.argmax(y_pred, -1), 4)
        for index in range(nclasses):
            iousum += new_iou(y_true[:,:,:,index], y_pred[:,:,:,index])
        return iousum/nclasses
    
    return mean_iou


def iou1(y_true, y_pred):
    y_pred = tf.one_hot(tf.argmax(y_pred, -1), 4)
    return new_iou(y_true[:,:,:,0], y_pred[:,:,:,0])

def iou2(y_true, y_pred):
    y_pred = tf.one_hot(tf.argmax(y_pred, -1), 4)
    return new_iou(y_true[:,:,:,1], y_pred[:,:,:,1])

def iou3(y_true, y_pred):
    y_pred = tf.one_hot(tf.argmax(y_pred, -1), 4)
    return new_iou(y_true[:,:,:,2], y_pred[:,:,:,2])

def iou4(y_true, y_pred):
    y_pred = tf.one_hot(tf.argmax(y_pred, -1), 4)
    return new_iou(y_true[:,:,:,3], y_pred[:,:,:,3])

In [8]:
model.compile(optimizer = 'adam', 
              loss =E_dice_coef_loss(nclasses), 
              metrics = [E_iou(nclasses), E_dice_multi_metric(nclasses), iou1, iou2,iou3,iou4]
             )


In [9]:

#lets now train the model! We can store all teh model output in 'history'.
# note that we are using the training generator for training data, the val 
# generator for validation data, we set the model to train for some number of epochs, 
# we tell teh network how many steps there are to get through the entire data set
# and we also give the network our callback to use.

results = model.evaluate(i_test,l_test)

4/4 [==============================] - 23s 2s/step - loss: 0.1833 - mean_iou: 0.7608 - dice_coef: 0.8062 - iou1: 0.9166 - iou2: 0.7912 - iou3: 0.8871 - iou4: 0.4483
